In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('/Users/gagan/Desktop/permafrost_data/Frac_Temp_Res_Tfrac_processed_timeseries1.csv')

In [5]:
data.head(3)

,Unnamed: 0,F02,F03,F04,F06,F18,F20,F21,T30_1,T30_2,...,Tfrac04_1,Tfrac04_2,Tfrac04_3,Tfrac04_4,Tfrac04_5,Tfrac07_1,Tfrac07_2,Tfrac07_3,Tfrac07_4,time
0,0,-1.1169,10.3142,6.5025,2.337562,27.1767,-4.6245,0.0006,-9.4934,-9.1609,...,-9.2480,-9.6308,-8.9031,-8.5332,-8.3830,-8.1888,-7.8033,-7.1370,-7.0778,2017-01-01T00:00:00Z
1,1,-1.1164,10.3174,6.5053,2.337520,27.1766,-4.6243,-0.0003,-9.5680,-9.2330,...,-9.1562,-9.3179,-8.7964,-8.4922,-8.3290,-8.4020,-7.9639,-7.1608,-7.0412,2017-01-01T01:00:00Z
2,2,-1.1145,10.3205,6.5107,2.337477,27.1758,-4.6229,-0.0008,-9.6568,-9.3084,...,-9.3568,-9.7409,-9.0686,-8.5397,-8.3888,-8.5605,-8.0196,-7.0832,-6.9179,2017-01-01T02:00:00Z


In [6]:
class rnn(nn.Module):
    def __init__(self, n_in = 52, n_out=1, nlin1 = 20, nlin2 = 20, n_rnn = 50):
        super(rnn, self).__init__()
        self.lin1 = nn.Linear(n_in, nlin1)
        self.lnorm1 = nn.LayerNorm(nlin1)
        self.gru = nn.GRUCell(input_size=nlin1, hidden_size=n_rnn)
        self.lin2 = nn.Linear(n_rnn, nlin2)
        self.lnorm2 = nn.LayerNorm(nlin2)
        self.lin3 = nn.Linear(nlin2, n_in)
        self.relu = nn.ReLU(True)
    def forward(self, x, h = None):
        x = self.relu(self.lnorm1(self.lin1(x)))
        h = self.gru(x, h)
        y = self.relu(self.lnorm2(self.lin2(h)))
        y = self.lin3(y)
        return y, h

# create train and test datasets and loaders

In [9]:
ntrain_samps = int(data.shape[0] * 0.7)
train_x = np.array(data.iloc[:ntrain_samps,:-1]).astype(float)
test_x = np.array(data.iloc[ntrain_samps:,:-1]).astype(float)
test_x.shape

train_dataset = TensorDataset(torch.from_numpy(train_x).float())
test_dataset = TensorDataset(torch.from_numpy(test_x).float())

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=False)

# create model

In [10]:
model = rnn(n_in=53)

In [1]:
xx = torch.FloatTensor(10, 53)
yy,h = model(xx)

NameError: name 'torch' is not defined

In [ ]:
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    criterion = nn.SmoothL1Loss()
    h = None
    for i,x in enumerate(train_loader):
        y, h = model(x, h)
        